In [55]:
import numpy as np
import pandas as pd
features = np.load('ABIDEII/data/features_batch1.npz')['x']
features2 = np.load('ABIDEII/data/features_batch2.npz')['x']
features3 = np.load('ABIDEII/data/features_batch3.npz')['x']
features4 = np.load('ABIDEII/data/features_batch4.npz')['x']
features5 = np.load('ABIDEII/data/features_batch5.npz')['x']
features6 = np.load('ABIDEII/data/features_batch6.npz')['x']
features = np.vstack((features,features2,features3,features4,features5,features6))
labels = np.load("ABIDEII/data/labels.npy")

In [56]:
features.shape

(403, 903)

In [57]:
len(labels)

403

In [58]:
# baseline dummy
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)

dummy_classifier_mf = DummyClassifier(strategy="most_frequent")
dummy_classifier_mf.fit(X_train, y_train)

y_hat = dummy_classifier_mf.predict(X_test)
acc_mf = accuracy_score(y_test, y_hat)
print(f"Dummy Accuracy score (most frequent) : {acc_mf}")

dummy_classifier_strat = DummyClassifier(strategy="stratified")
dummy_classifier_strat.fit(X_train, y_train)

y_hat = dummy_classifier_strat.predict(X_test)
acc_strat = accuracy_score(y_test, y_hat)
print(f"Dummy Accuracy score (stratified) : {acc_strat}")

Dummy Accuracy score (most frequent) : 0.45544554455445546
Dummy Accuracy score (stratified) : 0.43564356435643564


In [59]:
### Implementing cross validation ###

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np

k = 5
kf = KFold(n_splits=k, random_state=None)

train_accs_logreg, test_accs_logreg = [], []
train_accs_svc, test_accs_svc = [], []
logreg_models, svc_models = [], []

for train_index , test_index in kf.split(features):
    # Data splitting
    X_train , X_test = np.take(features,train_index,axis=0), np.take(features,test_index,axis=0)
    y_train , y_test = np.take(labels,train_index), np.take(labels,test_index)

    # Training on Logistic Regression
    logreg_model = LogisticRegression(solver='liblinear', penalty='l1')
    logreg_model.fit(X_train,y_train)
    pred_values_train = logreg_model.predict(X_train)
    acc_train = accuracy_score(y_train, pred_values_train)
    train_accs_logreg.append(acc_train)

    # Testing on Logistic Regression
    pred_values_test = logreg_model.predict(X_test)
    acc_test = accuracy_score(y_test, pred_values_test)
    test_accs_logreg.append(acc_test)


    # Training on Support Vector Classifier
    svc_model = SVC(kernel="linear", C=2000)
    svc_model.fit(X_train, y_train)
    acc_train = svc_model.score(X_train, y_train)
    train_accs_svc.append(acc_train)

    # Testing on Support Vector Classifier
    acc_test = svc_model.score(X_test, y_test)
    test_accs_svc.append(acc_test)


    # Saving each fold's model
    logreg_models.append(logreg_model)
    svc_models.append(svc_model)



print("### Logistic Regression results ###")
print(f"Training accuracy of each fold - {train_accs_logreg}")
print(f"Avg train accuracy : {sum(train_accs_logreg)/k}")
print(f"Testing accuracy of each fold - {test_accs_logreg}")
print(f"Avg testing accuracy : {sum(test_accs_logreg)/k}")
print()

print("### SVC results ###")
print(f"Training accuracy of each fold - {train_accs_svc}")
print(f"Avg train accuracy : {sum(train_accs_svc)/k}")
print(f"Testing accuracy of each fold - {test_accs_svc}")
print(f"Avg testing accuracy : {sum(test_accs_svc)/k}")
print()

best_logreg = logreg_models[np.argmax(test_accs_logreg)]
best_svc = svc_models[np.argmax(test_accs_svc)]

### Logistic Regression results ###
Training accuracy of each fold - [0.9968944099378882, 1.0, 1.0, 0.9969040247678018, 0.9969040247678018]
Avg train accuracy : 0.9981404918946983
Testing accuracy of each fold - [0.4444444444444444, 0.4567901234567901, 0.48148148148148145, 0.45, 0.6125]
Avg testing accuracy : 0.4890432098765432

### SVC results ###
Training accuracy of each fold - [1.0, 1.0, 1.0, 1.0, 1.0]
Avg train accuracy : 1.0
Testing accuracy of each fold - [0.4074074074074074, 0.43209876543209874, 0.5679012345679012, 0.475, 0.5375]
Avg testing accuracy : 0.4839814814814815

